In [ ]:
# import pandas
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from sklearn import preprocessing
from sklearn import tree
from kafka import KafkaProducer
from kafka.errors import KafkaError
from joblib import dump, load
import numpy as np
counter=0

In [ ]:

df = pd.read_csv('./csv_directory/Data'+str(counter)+'.csv')
df.head()
print(df.shape)
df.replace(to_replace = 7.0, 
                 value =3.0)
df.astype({'Column13': 'float'}).dtypes
# df.astype({'Column12': 'float'}).dtypes
num_of_classes = len(df.Column14.unique())
print(num_of_classes)


In [ ]:
# split train input and output data
X = df.drop(axis=0, columns=['Column12','Column14'])
Y = df.Column14

#Print the shape of X and Y
print(X.shape)
print(Y.shape)

In [ ]:
# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.30, random_state=42)

# Create a classifier
xgb = XGBClassifier(booster='gbtree', objective='multi:softprob', random_state=42, eval_metric="auc", num_class=num_of_classes)

# Fit the classifier with the training data
xgb.fit(X_train,y_train)

In [ ]:
# Use trained model to predict output of test dataset
val = xgb.predict(X_test)

lb = preprocessing.LabelBinarizer()
lb.fit(y_test)

y_test_lb = lb.transform(y_test)
val_lb = lb.transform(val)

roc_auc_score(y_test_lb, val_lb, average='macro')

In [ ]:
dump(xgb, './trained_models/model'+str(counter)+'.joblib') 
producer = KafkaProducer(bootstrap_servers=['localhost:9092'])
producer.send('trained_model',  bytearray('./trained_models/model'+str(counter)+'.joblib', 'utf-8'))
counter+=1